# Семинар 4. Линейная регрессия

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", palette="Spectral")

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

В прошлый раз мы выяснили, что в случае обучения с учителем будем иметь дело с матрицей объекты — признаки $ X $ и вектором целевых переменных $ y $:

$$
X = 
\begin{bmatrix}
x_{11} & x_{12} & \dots & x_{1d} \\
x_{21} & x_{22} & \dots & x_{2d} \\
\vdots & \vdots & \ddots & \vdots \\
x_{n1} & x_{n2} & \dots & x_{nd} \\
\end{bmatrix},
\quad
y = 
\begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_n
\end{bmatrix}.
$$

При этом для предсказания метки $ y \in \mathbb{R} $ неизвестного объекта мы использовали среднее значение меток его ближайших соседей. Сегодня мы познакомимся с новым классом моделей — линейными моделями. Для предсказания они используют линейную комбинацию признаков:

$$ \bar{y} = w_0 + w_1 x_1 + w_2 x_2 + \dots + w_d x_d = w_0 + \langle \mathbf{w}, \mathbf{x} \rangle, $$

где $ \mathbf{w} = (w_1, w_2, \dots, w_d) $ — вектор коэффициентов (весов) модели (без $ w_0 $), а $ \mathbf{x} = (x_1, x_2, \dots, x_d) $ — вектор признаков одного объекта.

Давайте для простоты представим, что $ w_0 $ соответствует константному признаку со значением 1 (т.е. добавим в $ X $ столбец из единиц). Тогда мы сможем внести $ w_0 $ в скалярное произведение:

$$ \bar{y} = \langle \mathbf{w}, \mathbf{x} \rangle. $$

В матричной форме для всего датасета (всех объектов) предсказания модели можно записать как:

$$ \bar{\mathbf{y}} = X \mathbf{w}. $$

Прочитаем данные из файла `train.csv`

In [ ]:
data = pd.read_csv("train.csv").sample(2_000, random_state=42)
print(f"{data.shape = }")
data.head()

Проверим, нет ли в данных пропущенных значений

In [ ]:
data.isna().sum()

Все значения в столбце `"id"` уникальны, давайте от него избавимся 

In [ ]:
data["id"].is_unique

In [ ]:
data = data.drop(columns=["id"])

Посмотрим на совместное распределение признаков и целевой переменной

In [ ]:
sns.pairplot(data=data)

In [ ]:
corr = data.corr(numeric_only=True)
corr

In [ ]:
sns.heatmap(corr)

А также построим гистограмму распределения целевой переменной

In [ ]:
sns.histplot(data=data, x="price", bins=20)

Разделим данные на обучающую и тестовую подвыборки

In [ ]:
X, y = data.drop(columns=["price"]), data["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train size: {y_train.size}")
print(f"Test size: {y_test.size}")

Наша модель умеет работать только с числовыми признаками

In [ ]:
NUMERIC_FEATURES = X.select_dtypes(include="number").columns
NUMERIC_FEATURES

Обучим модель и оценим ее качество с помощью RMSE (Root Mean Squared Error)

$$
\text{RMSE} = \sqrt{\frac{1}{n} \sum (y_i - \bar{y}_i)^2}
$$

In [ ]:
model = LinearRegression()
model.fit(X_train[NUMERIC_FEATURES], y_train)

y_pred = model.predict(X_test[NUMERIC_FEATURES])
print(f"Test RMSE: {root_mean_squared_error(y_test, y_pred):.3f}")

Посмотрим на распределение весов

In [ ]:
sns.barplot(x=model.coef_, y=model.feature_names_in_)

Можно ли утверждать, что признаки с большим значением весов важнее для нашей модели? Почему?

In [ ]:
scaler = StandardScaler()

X_train[NUMERIC_FEATURES] = scaler.fit_transform(X_train[NUMERIC_FEATURES])
X_test[NUMERIC_FEATURES] = scaler.transform(X_test[NUMERIC_FEATURES])

In [ ]:
model = LinearRegression()
model.fit(X_train[NUMERIC_FEATURES], y_train)

y_pred = model.predict(X_test[NUMERIC_FEATURES])
print(f"Test RMSE: {root_mean_squared_error(y_test, y_pred):.3f}")

In [ ]:
sns.barplot(x=model.coef_, y=model.feature_names_in_)

Теперь займемся категориальными признаками

In [ ]:
CATEGORICAL_FEATURES = X.columns.difference(NUMERIC_FEATURES)
CATEGORICAL_FEATURES

In [ ]:
one_hot_encoder = OneHotEncoder().fit(X_train[CATEGORICAL_FEATURES])
one_hot_encoder.get_feature_names_out()

In [ ]:
one_hot_encoder.transform(X_train[CATEGORICAL_FEATURES]).toarray()

In [ ]:
X_train[one_hot_encoder.get_feature_names_out()] = one_hot_encoder.transform(X_train[CATEGORICAL_FEATURES]).toarray()
X_train = X_train.drop(columns=CATEGORICAL_FEATURES)

X_test[one_hot_encoder.get_feature_names_out()] = one_hot_encoder.transform(X_test[CATEGORICAL_FEATURES]).toarray()
X_test = X_test.drop(columns=CATEGORICAL_FEATURES)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f"Test RMSE: {root_mean_squared_error(y_test, y_pred):.3f}")

Существует и более удобный способ предобработать данные — использовать `ColumnTransformer`.

In [ ]:
X, y = data.drop(columns=["price"]), data["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
column_transformer = ColumnTransformer(
    transformers=[
        ("numeric", StandardScaler(), NUMERIC_FEATURES),
        ("categorical", OneHotEncoder(), CATEGORICAL_FEATURES),
    ]
)

X_train_transformed = column_transformer.fit_transform(X_train)
X_test_transformed = column_transformer.transform(X_test)

Кроме того, мы можем обернуть все этапы построения модели в один `Pipeline`.

In [ ]:
pipeline = Pipeline(
    steps=[
        ("transformer", column_transformer),
        ("model", LinearRegression())
    ]
)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print(f"Test RMSE: {root_mean_squared_error(y_test, y_pred):.3f}")

### Регуляризация

In [ ]:
# TODO: reimplement
np.random.seed(451)

x = np.linspace(0, 1, 100)
y = np.cos(1.5 * np.pi * x)

x_objects = np.random.uniform(0, 1, size=30)
y_objects = np.cos(1.5 * np.pi * x_objects) + np.random.normal(scale=0.1, size=x_objects.shape)

fig, axs = plt.subplots(figsize=(16, 4), ncols=3)

for i, degree in enumerate([1, 4, 20]):
    X_objects = PolynomialFeatures(degree, include_bias=False).fit_transform(x_objects[:, None])
    X = PolynomialFeatures(degree, include_bias=False).fit_transform(x[:, None])
    regr = LinearRegression().fit(X_objects, y_objects)
    y_pred = regr.predict(X)
    axs[i].plot(x, y, label="Real function")
    axs[i].scatter(x_objects, y_objects, label="Data")
    axs[i].plot(x, y_pred, label="Prediction")

    if i == 0:
        axs[i].legend()
    
    axs[i].set_title("Degree = %d" % degree)
    axs[i].set_xlabel("$x$")
    axs[i].set_ylabel("$f(x)$")
    axs[i].set_ylim(-2, 2)

L1-регуляризация (Lasso)

$$ L = ... + \lambda \sum_{j=1}^{d} |w_j| $$

L2-регуляризация (Ridge)

$$ L = ... + \lambda \sum_{j=1}^{d} w_j^2 $$

Elastic Net

$$ L = ... + \lambda_1 \sum_{j=1}^{d} |w_j| + \lambda_2 \sum_{j=1}^{d} w_j^2 $$

In [ ]:
ridge = GridSearchCV(
    Ridge(max_iter=10_000),
    {"alpha": np.arange(0.1, 5.0, 0.25)},
    scoring="neg_mean_squared_error",
    cv=10,
    n_jobs=-1,
)
ridge.fit(X_train_transformed, y_train)

y_pred = ridge.predict(X_test_transformed)
print(f"Test RMSE: {root_mean_squared_error(y_test, y_pred):.2f}")

In [ ]:
ridge.best_estimator_.coef_

In [ ]:
lasso = GridSearchCV(
    Lasso(max_iter=10_000),
    {"alpha": np.arange(0.1, 5.0, 0.25)},
    scoring="neg_mean_squared_error",
    cv=10,
    n_jobs=-1,
)
lasso.fit(X_train_transformed, y_train)

y_pred = lasso.predict(X_test_transformed)
print(f"Test RMSE: {root_mean_squared_error(y_test, y_pred):.2f}")

In [ ]:
lasso.best_estimator_.coef_